In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

# S.11 빈도 분석
**명목변수 Categorical or nominal varaible**는 이름이 붙여진 변수로서 문자로 표현되는 변수를 말한다. 명목변수별로 빈도를 계산하고 차이가 있는지 보게 된다.

- 다른 사람의 빈도와 어떤 차이가 있는가?

In [1]:
names = ["kim","lee","park","lim"]
items = ["espresso","latte","americano","affocato","long black","macciato"]

In [3]:
# names에서 하나씩, items에서도 하나씩 추출
coffeeDf = spark.createDataFrame([(names[i%4], items[i%6]) for i in range(500)],\
                           ["name","coffee"])

In [19]:
coffeeDf.show()

+----+----------+
|name|    coffee|
+----+----------+
| kim|  espresso|
| lee|     latte|
|park| americano|
| lim|  affocato|
| kim|long black|
| lee|  macciato|
|park|  espresso|
| lim|     latte|
| kim| americano|
| lee|  affocato|
|park|long black|
| lim|  macciato|
| kim|  espresso|
| lee|     latte|
|park| americano|
| lim|  affocato|
| kim|long black|
| lee|  macciato|
|park|  espresso|
| lim|     latte|
+----+----------+
only showing top 20 rows



명목변수에 대해서는 평균, 표준편차와 같은 통계량을 계산하는 것은 의미가 없다.

In [4]:
coffeeDf.describe().show()

+-------+----+--------+
|summary|name|  coffee|
+-------+----+--------+
|  count| 500|     500|
|   mean|null|    null|
| stddev|null|    null|
|    min| kim|affocato|
|    max|park|macciato|
+-------+----+--------+



또는 문자열에 대해서 통계량을 구해보자. 개수, 키개수 unique, 첫째 값 top, 최빈키의 빈도 freq를 출력하고 있다.

In [5]:
coffeeDf.toPandas().describe()

,name,coffee
count,500,500
unique,4,6
top,lim,latte
freq,125,84


## S.11.1 crosstab
앞서 생성했던 데이터를 사용해서, 교차표를 만들어 보자.


pivot으로 건수를 계산하면 아래와 같다.

In [6]:
coffeeDf.groupBy('name').pivot('coffee').count().show()

+----+--------+---------+--------+-----+----------+--------+
|name|affocato|americano|espresso|latte|long black|macciato|
+----+--------+---------+--------+-----+----------+--------+
|park|    null|       42|      42| null|        41|    null|
| lim|      42|     null|    null|   42|      null|      41|
| kim|    null|       41|      42| null|        42|    null|
| lee|      41|     null|    null|   42|      null|      42|
+----+--------+---------+--------+-----+----------+--------+



crosstab 교차표를 만들면 pivot과 유사한 결과를 얻을 수 있다.  
대신 null값이 0으로 출력

In [7]:
coffeeDf.stat.crosstab("name", "coffee").show()

+-----------+--------+---------+--------+-----+----------+--------+
|name_coffee|affocato|americano|espresso|latte|long black|macciato|
+-----------+--------+---------+--------+-----+----------+--------+
|        lim|      42|        0|       0|   42|         0|      41|
|        lee|      41|        0|       0|   42|         0|      42|
|       park|       0|       42|      42|    0|        41|       0|
|        kim|       0|       41|      42|    0|        42|       0|
+-----------+--------+---------+--------+-----+----------+--------+



## S.11.2 freqItems()
데이터에 대해 50%이상 발생한 행을 출력해 보자. 임계치 이하의 최빈값을 구할 수 있다. (그닥 효력을 발휘하지 못하고 있다)

In [8]:
freq = coffeeDf.stat.freqItems(["name","coffee"], 0.5)
freq.show()

+--------------+-----------------+
|name_freqItems| coffee_freqItems|
+--------------+-----------------+
|   [lim, park]|[latte, espresso]|
+--------------+-----------------+



## S.11.3 Chi Square
Chi-Square 테스트는 label와 features 서로 상관관계가 있는지 검증하는 비모수 추정방법이다.   

**feature의 발생이 label과 어떤 관련이 있는지 추정**하는 것이다. 단 label, features 모두 명목변수이어야 한다.

- 귀무 가설은 label과 features 간에 서로 독립적이다.


- features 속성
- lable: 우리가 맞추려고 하는 값


- 키 몸무게(feature)->성별(lable)

#### label을 수로 변환
label, features 모두 명목변수이지만, 문자가 아니라 수와 벡터로 각 각 변환되어야 한다.

In [9]:
from pyspark.ml.feature import StringIndexer
#  이름을 숫자로 -> StringIndexer, index: 이름, output:라벨
labelIndexer = StringIndexer(inputCol="name", outputCol="label")
model=labelIndexer.fit(coffeeDf)
_coffeeDf=model.transform(coffeeDf)

In [10]:
_coffeeDf.printSchema() #label이 자동으로 double로 되어있다!

root
 |-- name: string (nullable = true)
 |-- coffee: string (nullable = true)
 |-- label: double (nullable = false)



In [11]:
_coffeeDf.show(4)

+----+---------+-----+
|name|   coffee|label|
+----+---------+-----+
| kim| espresso|  0.0|
| lee|    latte|  1.0|
|park|americano|  3.0|
| lim| affocato|  2.0|
+----+---------+-----+
only showing top 4 rows



#### features를 벡터로 변환
우선 명목 문자열을 수로 변환한다.

In [12]:
featureIndexer = StringIndexer(inputCol="coffee", outputCol="_features")
model=featureIndexer.fit(_coffeeDf)
_coffeeDf=model.transform(_coffeeDf)

다음은 수를 벡터로 변환한다. inputCols에는 리스트를 적어준다. 즉 "_features" (x) -> [ "_features" ] (o)이라고 한다.

In [13]:
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler #숫자로 놔두면 안되고 벡터로 만들어주어야한다!

va = VectorAssembler(inputCols=["_features"], outputCol="features")
_coffeeDf = va.transform(_coffeeDf)

In [14]:
_coffeeDf.printSchema()
_coffeeDf.show(5)

root
 |-- name: string (nullable = true)
 |-- coffee: string (nullable = true)
 |-- label: double (nullable = false)
 |-- _features: double (nullable = false)
 |-- features: vector (nullable = true)

+----+----------+-----+---------+--------+
|name|    coffee|label|_features|features|
+----+----------+-----+---------+--------+
| kim|  espresso|  0.0|      0.0|   [0.0]|
| lee|     latte|  1.0|      1.0|   [1.0]|
|park| americano|  3.0|      3.0|   [3.0]|
| lim|  affocato|  2.0|      2.0|   [2.0]|
| kim|long black|  0.0|      4.0|   [4.0]|
+----+----------+-----+---------+--------+
only showing top 5 rows



#### ChiSquareTest
ChiSquareTest에는 다음 인자를 적어준다.

- 데이터: 명목 label, 명목 features로 구성된 DataFrame.
- featuresCol: features 컬럼명을 적어준다. Vector 타입이어야 한다.
- labelCol: label 컬럼명을 적어준다. 숫자인 타입이면 된다.

In [15]:
from pyspark.ml.stat import ChiSquareTest
r = ChiSquareTest.test(_coffeeDf, "features", "label")

In [16]:
r.show()

+-------+----------------+-------------------+
|pValues|degreesOfFreedom|         statistics|
+-------+----------------+-------------------+
|  [0.0]|            [15]|[500.0963855421687]|
+-------+----------------+-------------------+



위 컬럼을 하나씩 출력해보자. pValues=0.0이므로 귀무가설을 기각한다.
- 서로 유의적인 상관관계가 있다.


In [17]:
from pyspark.ml.stat import ChiSquareTest
r = r.head()

In [18]:
print("pValues: " + str(r.pValues)) # 컬럼 개별적으로 출력 가능
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

pValues: [0.0]
degreesOfFreedom: [15]
statistics: [500.0963855421687]
